In [1]:
from sage.crypto.sboxes import Ascon, PRESENT, AES
from sage.geometry.polyhedron.constructor import Polyhedron
from sage.crypto.sbox import SBox
import random
from p_tqdm import p_map
from tqdm import tqdm
from collections import Counter

# Basic Sbox Properties

In [2]:
Ascon.differential_branch_number()

3

In [3]:
Ascon.linear_branch_number()

3

In [4]:
Ascon.differential_uniformity()

8

In [5]:
Ascon.maximal_linear_bias_absolute()

8

In [6]:
Ascon.boomerang_uniformity()

16

In [7]:
def frequency_count(frequency, table):
    count = 0
    for row in list(table):
        count += list(row).count(frequency)
    return count

In [8]:
print("DDT")
for freq in range(0, 9, 2):
    print(f"{freq} - {frequency_count(freq, Ascon.difference_distribution_table())}")
    
print("\nLAT")
for freq in range(-8, 9, 2):
    print(f"{freq} - {frequency_count(freq, Ascon.linear_approximation_table())}")
    
print("\nBCT")
for freq in range(0, 17, 2):
    print(f"{freq} & {frequency_count(freq, Ascon.boomerang_connectivity_table())} \\\\")
    print("\hline")

DDT
0 - 707
2 - 176
4 - 120
6 - 0
8 - 20

LAT
-8 - 18
-6 - 0
-4 - 174
-2 - 0
0 - 647
2 - 0
4 - 162
6 - 0
8 - 22

BCT
0 & 445 \\
\hline
2 & 176 \\
\hline
4 & 150 \\
\hline
6 & 0 \\
\hline
8 & 110 \\
\hline
10 & 0 \\
\hline
12 & 50 \\
\hline
14 & 0 \\
\hline
16 & 30 \\
\hline


<>:12: DeprecationWarning: invalid escape sequence '\h'
<>:12: DeprecationWarning: invalid escape sequence '\h'
/tmp/ipykernel_346/282164531.py:12: DeprecationWarning: invalid escape sequence '\h'
  print("\hline")


## 1-1 DDT, 1-1 LAT

In [9]:
def bit_1_1(sbox):
    if type(sbox) == list:
        sbox = SBox(sbox)
    elif type(sbox) != sage.crypto.sbox.SBox:
        raise TypeError("Sbox neither of type list nor sage.crypto.sbox.SBox")
        
    ddt = sbox.difference_distribution_table()
    lat = sbox.linear_approximation_table()
    bit_lat = []
    bit_ddt = []
    for i in range(sbox.m):
        bit_ddt.append([ddt[1 << i][1 << j] for j in range(sbox.m)])
        bit_lat.append([lat[1 << i][1 << j] for j in range(sbox.m)])
        
    return bit_ddt, bit_lat

In [10]:
bit_1_1(Ascon)

([[0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0]])

## BDT

In [11]:
Ascon_inv = Ascon.inverse()

In [12]:
bdt = [[[0] * 32 for i in range(32)] for i in range(32)]


for diff_0 in range(32):
    for diff_1 in range(32):
        for back_diff_0 in range(32):
            for x in range(32):
                if Ascon_inv[Ascon[x] ^^ back_diff_0] ^^ Ascon_inv[Ascon[x ^^ diff_0] ^^ back_diff_0] == diff_0:
                    if Ascon[x] ^^ Ascon[x ^^ diff_0] == diff_1:
                        bdt[diff_0][diff_1][back_diff_0] += 1

In [13]:
count = Counter()
for row in bdt:
    for col in row:
        count.update(col)
count

Counter({0: 31624, 4: 720, 2: 352, 8: 40, 32: 32})

In [14]:
bct = Ascon.boomerang_connectivity_table()

def latex_table(table):
    print("|c|" + "c"*32 + "|")
    print(" & ".join([f"{i:x}" for i in range(32)]) + "\\\\")
    print("\\hline")
    for i, row in enumerate(list(table)):
        print(f"{i} & " + " & ".join([str(value) if value != 0 else "$\\cdot$" for value in list(row)]) + "\\\\")
    print("\\hline")

## Convex Hull

In [15]:
def generate_vertices(sbox):
    ddt = sbox.difference_distribution_table()
    vertices = []
    for row in range(2**sbox.m):
        for col in range(2**sbox.m):
            if ddt[row][col]:
                vertices.append(list(map(int, list(f"{bin(row)[2:].zfill(sbox.m)}{bin(col)[2:].zfill(sbox.m)}"))))
    return vertices

In [16]:
ascon_hull = list(Polyhedron(vertices=generate_vertices(Ascon)).inequality_generator())

ascon_hull

[An inequality (-1, 0, 0, 0, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (0, -1, 0, 0, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (0, 0, -1, 0, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (0, 0, 0, -1, 0, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (-1, 0, 0, -1, -1, 0, 0, -1, -1, 0) x + 4 >= 0,
 An inequality (0, -1, 0, -1, -1, 0, 1, 1, 1, 0) x + 2 >= 0,
 An inequality (0, 0, 0, 0, -1, 0, 0, 0, 0, 0) x + 1 >= 0,
 An inequality (1, -1, -1, -1, -1, 0, 1, 0, 0, 0) x + 3 >= 0,
 An inequality (0, 0, 0, 0, 0, 0, 1, 0, 0, 0) x + 0 >= 0,
 An inequality (-1, 0, 0, -1, -1, 0, 0, 1, 1, 0) x + 2 >= 0,
 An inequality (0, 0, -1, -1, -1, 0, 1, 1, 1, 0) x + 2 >= 0,
 An inequality (-3, -2, -3, -4, -6, -1, -3, -3, -3, -1) x + 23 >= 0,
 An inequality (-1, -1, -1, -1, -1, 0, -1, 0, 0, 0) x + 5 >= 0,
 An inequality (-1, -1, -1, -2, -2, 0, -1, -1, -1, 0) x + 8 >= 0,
 An inequality (-1, -1, -1, -1, 1, 0, 1, 0, 0, 0) x + 3 >= 0,
 An inequality (0, -1, -1, -1, -1, 0, 0, -1, -1, 0) x + 5 >= 0,
 An inequality

In [17]:
len(ascon_hull)

2415

## Sboxes - LuT, ANF, K-Map

In [18]:
def anf(s_box):
    result = []
    for i in range(0, s_box.m):
        result.append(
            [term.split("*") for term in str(s_box.component_function(1 << i).algebraic_normal_form()).split(" + ")]
        )
    return result

In [19]:
kmap = """y[0] = x[4] x[3] x[1]  + x[4]' x[3] x[1]'  + x[3]' x[1]' x[0] + x[3]' x[1] x[0]';
y[1] = x[4] x[3] x[2]  + x[4] x[3]' x[2]'  + x[3] x[2] x[1]' x[0] + x[3]' x[2]' x[1]' x[0] + x[3] x[2] x[1] x[0]' + x[3]' x[2]' x[1] x[0]' + x[4]' x[3]' x[2] x[1] x[0] + x[4]' x[3] x[2]' x[1] x[0] + x[4]' x[3]' x[2] x[1]' x[0]' + x[4]' x[3] x[2]' x[1]' x[0]';
y[2] = x[3] x[2] x[1]  + x[3]' x[2]' x[1]  + x[3] x[2] x[0]' + x[3]' x[2]' x[0]' + x[3]' x[2] x[1]' x[0] + x[3] x[2]' x[1]' x[0];
y[3] = x[4]' x[3] x[2] x[1] x[0] + x[4]' x[3]' x[2]' x[1]' x[0] + x[4] x[3] x[2] x[1] x[0]' + x[4] x[3]' x[2]' x[1]' x[0]' + x[4] x[3]' x[2] x[0] + x[4] x[2]' x[1] x[0] + x[4] x[3] x[1]' x[0] + x[4]' x[3]' x[2] x[0]' + x[4]' x[2]' x[1] x[0]' + x[4]' x[3] x[1]' x[0]';
y[4] = x[3] x[1] x[0] + x[3] x[1]' x[0]' + x[4] x[3]' x[2] x[1]  + x[4]' x[3]' x[2]' x[1]  + x[4]' x[3]' x[2] x[1]'  + x[4] x[3]' x[2]' x[1]';"""


In [20]:
def verilog_lut_sbox(sbox, export_file="sbox"):
    """
    Takes a n-bit S-Box as input and generates a Verilog script file
    implementing the S-Box.
    """
    # Define the Verilog module for the S-Box.
    module = f"""module {export_file}(input [{len(sbox) - 1}:0] x, output reg [{len(sbox) - 1}:0] y);
    always @(*)
        case (x)
"""
    for s_in in range(2 ** len(sbox)):
        module += f"\t\t{len(sbox)}'d{s_in}: y = {len(sbox)}'d{sbox[s_in]};\n"
    module += f"""
            default: y = 5'd0;
        endcase
endmodule"""
    
    # Write the Verilog module to a file.
    with open(f"{export_file}.v", "w") as f:
        f.write(module)
        
    print(f"Verilog S-Box module written to {export_file}.v")

In [30]:
def verilog_anf_sbox(anf, export_file="sbox"):
    """
    Takes a n-bit S-Box in SageMath lgebraic normal form as input and
    generates a Verilog script file implementing the S-Box in algebraic
    normal form.
    """
    # Define the Verilog module for the S-Box.
    module = f"""module {export_file}(input [{len(anf) - 1}:0] x, output reg [{len(anf) - 1}:0] y);
    always @(x)
    begin
    """

    # Generate the Verilog code for the S-Box.
    for y_bit in range(len(anf)):
        module += f"\ty[{y_bit}] <= "
        for term_index, term in enumerate(anf[y_bit]):
            if term_index == 0:
                module += "("
            else:
                module += ") ^ ("
            for x_index, x_bit in enumerate(term):
                if x_index == 0 and len(x_bit) == 2:
                    module += f"x[{x_bit[-1]}]"
                elif len(x_bit) == 2:
                    module += f" & x[{x_bit[-1]}]"
                else:
                    module += x_bit
            if term_index == len(anf[y_bit]) - 1:
                module += ");\n\t"

    module += "end\nendmodule"

    # Write the Verilog module to a file.
    with open(f"{export_file}.v", "w") as f:
        f.write(module)

    print(f"Verilog S-Box module in ANF written to {export_file}.v")

In [21]:
def verilog_kmap_sbox(logic, export_file="sbox"):
    lines = [line.split("=")[1].strip() for line in logic.split("\n") if line]
    # Define the Verilog module for the S-Box.
    module = f"""module {export_file}(input [{len(lines) - 1}:0] x, output [{len(lines) - 1}:0] y);
begin
"""
    # Generate the Verilog code for the S-Box.
    for y_bit, line in enumerate(lines):
        module += f"\tassign y[{y_bit}] = ("
        new_terms = []
        for term in [t.strip() for t in line.split("+") if t]:
            new_term = [x_bit if "'" not in x_bit else "!" + x_bit.replace("'", "") for x_bit in term.split()]
            new_terms.append(" & ".join(new_term))
        new_terms[-1] = new_terms[-1][:-1] + ");\n" 
        module += ") | (".join(new_terms)
    
    module += "end\nendmodule"

    # Write the Verilog module to a file.
    with open(f"{export_file}.v", "w") as f:
        f.write(module)

    print(f"Verilog S-Box module in KMAP written to {export_file}.v")


In [22]:
def generate_tcl(export_file="sbox"):
    script = f"""set_db lib_search_path ..
set_db library analysis.lib
set_db hdl_search_path /
read_hdl {export_file}.v
elaborate {export_file}
synthesize -to_mapped -effort medium
write_hdl > {export_file}_netlist.v
report gates > {export_file}.rep
exit"""

    # Write the tcl script to a file.
    with open(f"{export_file}.tcl", "w") as f:
        f.write(script)

    print(f"TCL Script written to {export_file}.tcl")

In [4]:
# # Generate Verilog Files for LUT and ANF
# verilog_lut_sbox(Ascon, "ascon_lut")
# verilog_anf_sbox(anf(Ascon), "ascon_anf")
# verilog_kmap_sbox(kmap, "ascon_kmap")

# # Generate TCL Script for Genus    
# generate_tcl("ascon_lut")
# generate_tcl("ascon_anf")
# generate_tcl("ascon_kmap")

generate_tcl("round_constant")
generate_tcl("linear")
generate_tcl("s_box_layer")
generate_tcl("permutation")

TCL Script written to round_constant.tcl
TCL Script written to linear.tcl
TCL Script written to s_box_layer.tcl
TCL Script written to permutation.tcl


# Zero - Sum

In [ ]:
def generate_inputs(bit_length, order):
    if bit_length < order:
        raise ValueError(f"Given Bit Length - {bit_length} is smaller than the Order of Derivative - {order}")
        
    variable_positions = sorted(random.sample(range(bit_length), order))
    constant = bin(random.randint(0, 2 ** (bit_length - order)))[2:].zfill(bit_length - order)
    inputs = []
    for var in tqdm(range(2 ** order), desc="Generating Inputs"):
        variable = bin(var)[2:].zfill(order)
        input_bin = list(constant)
        
        for index, position in enumerate(variable_positions):
            if position < len(input_bin):
                input_bin.insert(position, variable[index])               
            else:
                input_bin.append(variable[index])
                
        inputs.append(int("".join(input_bin), 2))
        
    return inputs

In [ ]:
def zero_sum(order, rounds=3):
    key = random.randint(0, 2**80)
    present = PRESENT(rounds=rounds) 
    input_sum = 0
    output_sum = 0
    inputs = generate_inputs(64, order)
    for input_val in tqdm(inputs, desc="Summing Inputs and Outputs"):
        input_sum ^^= input_val
        output_sum ^^=  present.encrypt(input_val, key)
    
    print(f"Order - {order}, Input Sum - {input_sum}, Output Sum - {output_sum}")